In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('missingData').getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/tables/empData.txt',inferSchema=True,header=True)

In [0]:
df.show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.2|
emp4|Cindy|456.8|
+----+-----+-----+

In [0]:
#dropping missing data
df.na.drop().show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.8|
+----+-----+-----+

In [0]:
#applying row threshold if row has certain nulls
df.na.drop(thresh=2).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp3| null|345.2|
emp4|Cindy|456.8|
+----+-----+-----+

In [0]:
#based on particular columns we can apply drop
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp3| null|345.2|
emp4|Cindy|456.8|
+----+-----+-----+

In [0]:
#it is smart enough and fill values based on data type compatability
df.na.fill('FillValue').show()

+----+---------+-----+
 Id| Name|Sales|
+----+---------+-----+
emp1| John| null|
emp2|FillValue| null|
emp3|FillValue|345.2|
emp4| Cindy|456.8|
+----+---------+-----+

In [0]:
df.na.fill(0).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| 0.0|
emp2| null| 0.0|
emp3| null|345.2|
emp4|Cindy|456.8|
+----+-----+-----+

In [0]:
#best practice is to apply subset to avoid data corruption
df.na.fill('NoName',subset=['Name']).show()

+----+------+-----+
 Id| Name|Sales|
+----+------+-----+
emp1| John| null|
emp2|NoName| null|
emp3|NoName|345.2|
emp4| Cindy|456.8|
+----+------+-----+

In [0]:
#common method to fill missing values with mean
from pyspark.sql.functions import mean

In [0]:
mean_values = df.select(mean(df['Sales'])).collect()
mean_values[0][0]

Out[22]: 401.0

In [0]:
df.na.fill(mean_values[0][0],['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John|401.0|
emp2| null|401.0|
emp3| null|345.2|
emp4|Cindy|456.8|
+----+-----+-----+